For a random matrix, the eigenvalue distribution follows in many cases the Wigner semicircle rule. With this one can calculate entropies, etc.

In [2]:
from pylab import *
import scipy.special as sp

# semicircular DOS - Wigner rule
def DOS( x):
    return 2.0*sqrt(1.0-x**2)/pi

# integrated DOS for semicircular DOS:
# \int_{-1}^{x} dy DOS(y) =
def Ifunc( x): 
    return 0.5+arcsin(x)/pi+x*sqrt(1.0-x*x)/pi

# an integral for E with semicircluar DOS:
# \int_{-1}^{x} dy DOS(y) y =
def Efunc( x): 
    return -2.0*((1.0-x*x)**(1.5))/(3.0*pi)

# set matrix dimension
N=2**6
    
# if uni=1 then uniform [-1,1] , otherwise standard normal
uni=2
if uni==1: # uniform [-1,1]
    A, sigmasqr, txt = 2.0*random(N*N) - 1.0, 1.0/3.0, 'uniform [-1,1]'
else:
    if uni==2: # sparse binomial
        A, sigmasqr, txt = [ round(x) for x in 2.0*random(N*N) - 1.0 ], 0.5, 'sparse binom.'
    else: # std normal
        A, sigmasqr, txt = normal(0.,1.,N*N), 1.0, 'std normal'

# eigenvalue bandwidth scales with sqrt(N 4 \sigma^2)
Enorm=sqrt(4.0*N*sigmasqr) 
# define H
H=zeros((N,N))
for i in range(N):
    for j in range(i): # we don't actually use all random values
        H[i,j]=A[i+N*j]
        H[j,i]=A[i+N*j]

# eigenvalues
evals=sorted(linalg.eigvals(H))

# "Gibbs" entropy, etc, for the given random instance
wg=arange(N)+1.
sg=log(wg)
eg=cumsum(evals)/wg

# "Gibbs" from the Wigner rule
eW=arange(-0.9999,1.,0.01)
wgW=Ifunc( eW)
egW=Efunc( eW)/Ifunc( eW)

# calculate canonical energy and entropy  for the given random instance
beta = arange(-2.999,3.,0.04)
betalambda = outer( beta, evals)
zbeta= sum( exp(-betalambda), axis=1)
ebeta= sum( exp(-betalambda)*betalambda, axis=1)
ebeta= ebeta/zbeta/beta
sbeta= log(zbeta)+beta*ebeta

# canonical energy and entropy with Wigner rule (using integral formula Gradstein Rhyzhik 3.387.1)
zbetaW=N*2.0*sp.iv(1, beta*Enorm)/beta/Enorm
ebetaW=1.0/beta - 0.5*Enorm*(sp.iv(0, beta*Enorm) +  sp.iv(2, beta*Enorm))/sp.iv(1,beta*Enorm)
sbetaW=log(zbetaW)+beta*ebetaW


# plotting
subplot(2,1,1)
plot(evals/Enorm, wg/N, 'ro-', label='(rand.)')
plot( eW, wgW, 'k-', label='(Wigner)')
xlim(-1.1,1.1)
xlabel(r'${\cal E}/\sqrt{N 4\sigma^2}$',size=16)
ylabel(r'$W_G({\cal E})/N$',size=16)

subplot(2,1,2)
plot(evals/Enorm, eg/Enorm, 'ro-', label='(rand.)')
plot( eW, egW, 'k-', label='(Wigner)')
xlim(-1.1,1.1)
ylim(-1.1,0.1)
xlabel(r'${\cal E}/\sqrt{N 4\sigma^2}$')
ylabel(r'$E({\cal E})/\sqrt{N 4\sigma^2}$')

savefig('randommatrix_Evals.png')

clf() # just clearing this plot


subplot(2,1,1)
plot(eg/Enorm, log(wg)/N, 'ro-', label=r'$S_{imc}$ (rand.)')
plot( egW, log(N*wgW)/N, 'k-', label=r'$S_{imc}$ (Wigner)')
plot( ebeta/Enorm, sbeta/N, 'bx-', label=r'$S_c$ (rand.)')
plot( ebetaW/Enorm, sbetaW/N, 'b--', label=r'$S_c$ (Wigner)')
xlabel(r'$E/\sqrt{N 4\sigma^2}$',size=16)
ylabel(r'$S/N$',size=16)
xlim(-1.,1.)
ylim(0.,1.1*log(N)/N)
legend(loc=8)
text(-0.95,1.0*log(N)/N,r'matrix size $%d\times%d$' % (N,N), size=16)
text(-0.95,0.9*log(N)/N, txt, size=16)

subplot(2,1,2)
plot(eg/Enorm, gradient(log(wg))/gradient(eg), 'ro-', label='imc (rand.)')
plot( egW, gradient(log(N*wgW))/gradient(Enorm*egW), 'k-', label='imc (Wigner)')
plot( ebeta/Enorm, beta, 'bx-', label='canonical (rand)')
plot( ebetaW/Enorm, beta, 'b--', label='canonical (Wigner)')
xlabel(r'$E/\sqrt{N 4\sigma^2}$',size=16)
ylabel(r'$1/k_B T$',size=16)
xlim(-1.,1.)
ylim(-1.5,1.5)
axvline( 0, color='k')
axhline( 0, color='k')

savefig('randommatrix.png')
gcf().canvas.set_window_title('Entropies and temperatures for random matrices')
show()


![caption](randommatrix.png "Random matrices")